In [6]:
import os
import datetime

import numpy as np
import pandas as pd
import pymc3 as pm
import scipy
import theano.tensor as tt

import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import plotly.plotly as py
import plotly.tools as tls

from IPython.core.display import display, HTML

plotly.tools.set_credentials_file(username=os.environ.get('PLOTLY_USERNAME'), api_key=os.environ.get('PLOTLY_KEY'))

# Summary

After receiving the data, I did a quick cursory look and tried to conduct a series of analyses that I felt would be most beneficial to **(omitted)** given their data. The following questions help framed what I am looking and examining for are:

   - What foods are usually paired with each other?
   - What relationship do the foods have to each other?
   - Is there a seasonality effect on food choices?
   
The first question looks at combinations of food that are ordered together. Since the transaction ids can have multiple items, it would be beneficial to see what items are bought with each other. This would be helpful if we are trying to upsell certain items or we can use to train employees to make recommendations. 

The second question looks at the relationship of the food combinations. Now that we know what kind of combinations of food are frequently bought together, what can we infer about their relationship? This would be useful if want to sell certain combinations at certain time or providing special deals for certain food combination.

The third question is to look at any temporal effects on our sales. Rather than looking at the combination of food sales being dependent on our ability to market to the customer, it may be due to external factors. Having used to work in multiple restaurants, I know that the weather or time of year can play a huge role in driving sales of certain items. 

In [2]:
data = pd.read_csv('/food_dataset.csv')
data.head()

,dateofpurch,item,transactionid
0,2014-04-27,Greens&Grains,1cb598ac-24ff-4734-ab8d-b4422c467870
1,2014-04-27,Chicken,1cb598ac-24ff-4734-ab8d-b4422c467870
2,2014-04-27,chips,1cb598ac-24ff-4734-ab8d-b4422c467870
3,2014-04-27,Salad,1cb598ac-24ff-4734-ab8d-b4422c467870
4,2014-04-27,braised_lamb,1cb598ac-24ff-4734-ab8d-b4422c467870


# Exploratory Analysis

Here I looked at some descriptive statistics. Soon, I saw that there was a date of transaction, so I transformed the column into a `datetime` format so I can do time operations down the road. 

I was curious to answer certain questions such as "What are the most common items sold?"

In [3]:
data['date'] = data['dateofpurch'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
data.item.value_counts()

Chicken          373191
Bowl             316073
Salad            270926
falafel          159859
meatballs        104021
cookie           100016
lamb_sliders      99115
Minis             90430
Pita              89763
Greens&Grains     89508
braised_lamb      67842
chips             63558
braised_beef      51731
soup              44982
large_drink       39218
small_drink       37797
Name: item, dtype: int64

Having queried this, I now wanted to know the number of items sold and unique transactions. I also looked at the standard deviations of the items and unique transaction a day to see the variability, if the standard deviation is high, we can expect the number sales a day not to be consistent and would warrant further investigation with questions such as "What specific day do we see higher or lower sales?" or "What kind of days would skew the sales data?" However, the standard deviation was pretty small, so I did not investigate that much further.

In [4]:
# number of purchases by day
data.dateofpurch.value_counts()
stdev_day = np.std(data.dateofpurch.value_counts().values)

# number of unique transaction by day
grouped = data.groupby(['dateofpurch', 'transactionid']).count().groupby(level='dateofpurch').size()
std_dev_trans = np.std(grouped.values)

print('Standard Deviation of number of items sold a day:', str(stdev_day))
print('Standard Deviation of number of unique transactions a day:', str(std_dev_trans))

Standard Deviation of number of items sold a day: 82.5111116985
Standard Deviation of number of unique transactions a day: 24.1223939028


In [96]:
def create_item_counts_by_date(data, item, full_index, date_column, food_column):
    sub = data.ix[data[food_column] == item, :]
    indx = sub.groupby(date_column).count().index
    vals = sub.groupby(date_column).count()[food_column].values
    item_dict = dict(zip(indx, vals))
    fill_in_dates = list(set(full_index) - set(indx))
    for date in fill_in_dates:
        item_dict[date] = 0
    return item_dict

def create_chart_by_items(data, items, date_column='dateofpurch', food_column='item', title='Sales'):
    graph = []
    for item in items:
        test = create_item_counts_by_date(data, item, set(data[date_column]), date_column, food_column)

        # Create traces
        trace = go.Scatter(
            x = sorted(set(data[date_column])),
            y = [i[1] for i in sorted(test.items())],
            mode = 'lines',
            name = item[0].upper() + item[1:]
        )
        graph.append(trace)

    layout = dict(title = title,
                  xaxis = dict(title = 'Month'),
                  yaxis = dict(title = 'Number of Items Sold'.format(item)),
                  )
    fig = dict(data=graph, layout=layout)
    return fig

After looking at some descriptive statistics, I created a short function that would parse the data and create a chart that visualize the sales by day, since I will be reusing it often during the explanatory analysis phase. Below are plots are the sales of different proteins and food bases by day.

In [23]:
# looking at the base items
# items = ['Chicken', 'meatballs', 'falafel', 'braised_lamb', 'braised_beef']
# fig = create_chart_by_items(data, items, title='Sales by Protein Type')
# py.iplot(fig, filename='protein')
html = '''
<iframe width="900" height="600" frameborder="0" scrolling="no" src="//plot.ly/~minh5/382.embed"></iframe>
'''
display(HTML(html))

In [22]:
# items = ['Salad', 'Greens&Grains', 'Bowl', 'Pita', 'Minis']
# fig = create_chart_by_items(data, items, title='Sale by Base Item')
# py.iplot(fig, filename='base')
html = '''
<iframe width="900" height="600" frameborder="0" scrolling="no" src="//plot.ly/~minh5/380.embed"></iframe>
'''
display(HTML(html))

From the graphs, there are a few patterns that emerges. First off, chicken remains the number one protein above all other protein types. In fact, there are no day from 2013 to 2015 where there were any protein sells more than chicken. Falafel is a distant second followed by meatball, braised lamb, and braised beef.  Second, it appears that Salads and Bowls are the top selling base items, with Bowls generally overselling Salads but the distinction is not as clear, since there are some days salads sell more than bowls. The other base types (pita, minis , and greens and grains) are mixed in together.

# What foods are usually paired with each other?

Now that I have a pretty good idea of the items sold. I started looking at what kind of transactions take place. From a quick count, we see that 75% of all transactions have 2 or 3 items. There is also a small number of people ordering 17 or more items on (perhaps a group meal).

In [7]:
# most common items purchases
grouped = data.groupby('transactionid').count()
grouped['item'].value_counts()

2     343713
3     148570
4      73338
5      38261
6      29566
7       9778
8       7498
9       3673
10      1652
11       957
12       543
1        532
13       220
14       147
15        72
16        35
17        22
19         7
18         5
20         3
21         1
23         1
Name: item, dtype: int64

In [8]:
# looing at tranactions where there are 2 or 3 
single_indx = grouped.ix[grouped['item'] == 1, :].index.values.tolist()
multiple_indx = grouped.ix[grouped['item'].isin([2,3,4]), :].index.values.tolist()

There are also 500 people who ordered only 1 item, which after a look quick, turns out to just be pitas.

In [10]:
multiple = data.ix[data['transactionid'].isin(multiple_indx), :]
def collapse_food_by_id(data, new_column_name='combo'):
    results_dict = {}
    for tid in data['transactionid']:
        results_dict[tid] = []
    for i, row in data.iterrows():
        results_dict[row['transactionid']] = sorted(results_dict[row['transactionid']] + [row['item']])
    results_dict = {k: (', ').join(v) for k, v in results_dict.items()}
    df = pd.DataFrame.from_dict(results_dict, orient='index')
    df.columns = [new_column_name]
    df['counts'] = df[new_column_name].apply(lambda x: x.count(',') + 1)
    return df

In [9]:
data.ix[data['transactionid'].isin(single_indx), 'item'].value_counts()

Pita    532
Name: item, dtype: int64

Next since a transaction can have multiple items associated with it, I must find a way to group the items together. To do this, I take each transaction and create a dictionary with the transaction id as the key and all the items in that transaction as the value. Then after creating the dictionary, I joined the list of items after sorting it (to make sure `["Bowl", "Chicken"]` is the same `["Chicken", "Bowl"]`) to make a string representation of the transaction. After that, you can simply transform a dictionary into a pandas DataFrame.

Here I can look at the most popular items sold. It turns out that the most popular item is a Chicken bowl, which makes sense because the most popular protein is chicken and the most popular base items are either the bowl or salad. 

In [11]:
df = collapse_food_by_id(multiple)
df.combo.value_counts()[0:10]

Bowl, Chicken             71236
Chicken, Salad            49540
Salad, falafel            41733
Chicken, Greens&Grains    28352
Bowl, falafel             24627
Bowl, meatballs           12666
Chicken, Pita             12107
Salad, meatballs          10772
Bowl, Chicken, cookie     10127
Pita, meatballs            9450
Name: combo, dtype: int64

During this process, I have noticed is that the `Bowl, Chicken` is actually one item. Originally, I looked at these two as separate items. From this, we can see that if the transaction has two items, it is actually one item. The count of two represents the base and the protein as one item, or an entree. So below, we can see the most popular entree for transactions where a customer only bought an entree. 

In [12]:
df.ix[df['counts'] == 2, :]['combo'].value_counts()[0:10]

Bowl, Chicken             71236
Chicken, Salad            49540
Salad, falafel            41733
Chicken, Greens&Grains    28352
Bowl, falafel             24627
Bowl, meatballs           12666
Chicken, Pita             12107
Salad, meatballs          10772
Pita, meatballs            9450
Minis, meatballs           9231
Name: combo, dtype: int64

In [21]:
merged = pd.merge(df, data.ix[:, ['transactionid', 'date']], left_index=True, right_on='transactionid')
items = merged.combo.value_counts().index.values.tolist()[0:5]
fig2 = create_chart_by_items(merged, items, date_column='date', food_column='combo', title='Sales of Combinations')
py.iplot(fig2, filename='combos')
html = '''
<iframe width="900" height="600" frameborder="0" scrolling="no" src="//plot.ly/~minh5/384.embed"></iframe>
'''
display(HTML(html))

# Is there a correlation between sales of certain items?

Given that the counts of the sales of entrees are kind of muddled. I would want to examine the relationship between different entrees. For instance, on days that chicken bowl sales are high, does that mean sales of chicken salads are low? Do people make a trade off between falafel and chicken bowls? The purpose of this is that we can focus on certain items and use it to predict sales. If we know that on certain days, salads are sold more, we can focus on stocking inventory of salads on those days.

To do this, I simply ran a quick Pearson's Correlation between the counts of certain items. The Pearson's R statistic gives a number between -1 and 1. Values closer to -1 means an inverse relationship, an increase in one item means a decrease in the other, and values closer to 1 means a direct relationship, where an increase in one item means an increase in the other item.

For instance, I looked at the correlation between the sales of Chicken bowl and their salad and Greens&Grains counterparts.

In [124]:
bowl_sales_by_day = merged.ix[merged['combo'] == 'Bowl, Chicken', :].groupby('date').count()['combo'].values.tolist()
salad_sales_by_day = merged.ix[merged['combo'] == 'Chicken, Salad', :].groupby('date').count()['combo'].values.tolist()
corr = scipy.stats.pearsonr(bowl_sales_by_day, salad_sales_by_day)[0]
print('correlation between Bowl and Salad: ', str(corr))

bowl_sales_by_day = merged.ix[merged['combo'] == 'Bowl, Chicken', :].groupby('date').count()['combo'].values.tolist()
salad_sales_by_day = merged.ix[merged['combo'] == 'Chicken, Greens&Grains', :].groupby('date').count()['combo'].values.tolist()
corr = scipy.stats.pearsonr(bowl_sales_by_day, salad_sales_by_day)[0]
print('correlation between Bowl and Greens&Grains: ', str(corr))

correlation between Bowl and Salad:  -0.0479223236549
correlation between Bowl and Greens&Grains:  -0.0310560640055


From here we can see there is a just a slight *negative* correlation between the different chicken items. The strength of the correlation is a bit weaker between the bowls and the greens&grains than the bowls and salad. From here, it looks like there is a bit of a trade off between sales of chicken bowls and the other chicken-based entree. Although it is important to keep in mind that correlations is not causation and that we would have to do more tests to test of causality.

However, I moved onto more questions about the data that may be more useful. Given that there are transactions of more than one entrees, I'd like to understand who are the people bought more than just the entree. There are two categories of this: customers who buy multiple entrees and customers who add side items.

# What are people buying when it's more than one meal?

People who purchase between one to two entrees make up the majority of transactions. While more customers bought single entrees, it may be useful to figure out purchasing patterns for transactions with more than one entree. For instance, if we know that people buy the same base and entrees, we could offer certain buy one, get one (BOGO) deals. Or if we wanted to attract more customers, we could entice them to join a current **(omitted)** customer. These behavior patterns could have postive business implications.

To figure this out, I just compared the counts of the items to the unique count of items in a transaction. The first output below shows transaction that are composed of different food bases and proteins, while the second one, it appears that they order two exact same entree combination.

In [131]:
merged['nunique'] = merged['combo'].apply(lambda x: len(set(x.replace(' ', '').split(','))))
merged.ix[((merged['counts'] == 4) & (merged['nunique'] == 4)), :][0:5]

,combo,counts,transactionid,date,nunique
244295,"Chicken, Greens&Grains, Minis, braised_lamb",4,e50e99d2-0d1b-4aa1-884c-3823c4acfa9f,2013-04-25,4
244296,"Chicken, Greens&Grains, Minis, braised_lamb",4,e50e99d2-0d1b-4aa1-884c-3823c4acfa9f,2013-04-25,4
244297,"Chicken, Greens&Grains, Minis, braised_lamb",4,e50e99d2-0d1b-4aa1-884c-3823c4acfa9f,2013-04-25,4
244298,"Chicken, Greens&Grains, Minis, braised_lamb",4,e50e99d2-0d1b-4aa1-884c-3823c4acfa9f,2013-04-25,4
1952838,"Bowl, Chicken, Minis, braised_lamb",4,4f611e17-0df6-49f3-8160-214a512a25d5,2013-06-26,4


In [132]:
merged.ix[((merged['counts'] == 4) & (merged['nunique'] == 2)), :][0:5]

,combo,counts,transactionid,date,nunique
727890,"Bowl, Bowl, Chicken, Chicken",4,033e2626-f073-4824-88c9-3f0f1001074d,2014-02-03,2
727891,"Bowl, Bowl, Chicken, Chicken",4,033e2626-f073-4824-88c9-3f0f1001074d,2014-02-03,2
727892,"Bowl, Bowl, Chicken, Chicken",4,033e2626-f073-4824-88c9-3f0f1001074d,2014-02-03,2
727893,"Bowl, Bowl, Chicken, Chicken",4,033e2626-f073-4824-88c9-3f0f1001074d,2014-02-03,2
285480,"Bowl, Bowl, falafel, falafel",4,dc48a646-0a52-4c59-97fd-11a275ea3d7c,2015-01-14,2


Below is the cross tab of the number of items and the unique number of items in a transaction. We see that there is a sizeable share of people who purchase the same entree combination, around 30%. We can use this information to attract new customers or entice current customers to frequent **(omitted)** more often with special offers.

In [130]:
pd.crosstab(merged['counts'], merged['nunique'])

nunique,1,2,3,4
counts,,,,
2,80,687346,0,0
3,6,42,445662,0
4,4,98836,80788,113724


# How often do people add side items?

On the other end of the spectrum, the people who ordered more than one entree are the ones who had add ons: drinks, cookies, or chips. To figure this out, I created a dummy variable that denotes a 1 if the transaction had an add on and 0 if there are no add ons. Here, we can take a quick look at the most frequent transaction with an add on.

Again, it is no surprise that the most popular item, chicken bowl, has an add on as well. However, it is surprising that lamb sliders show up more often. It could be that lamb sliders are considered an add on (I did not consider them as an add on when creating the variable) or that people are more likely to purchase add ons when buying lamb sliders. Furthermore, it is important to note that falafel and meatball protein choices are not in the top ten transaction with add ons. 

In [137]:
add_ons = ['chips', 'cookie', 'large_drink', 'small_drink', 'soup']
merged['add_ons'] = merged['combo'].apply(lambda x: 1 if any([a in x for a in add_ons]) else 0)
merged.ix[merged['add_ons'] == 1, :]['combo'].value_counts()[0:10]

Bowl, Chicken, cookie             30381
Chicken, Salad, cookie            20955
Bowl, Chicken, soup               13539
Bowl, chips, lamb_sliders         13377
Chicken, Greens&Grains, cookie    12087
Salad, chips, lamb_sliders        11595
Bowl, braised_lamb, chips         10932
Chicken, Salad, soup               9720
Pita, chips, lamb_sliders          9618
Bowl, braised_lamb, cookie         8127
Name: combo, dtype: int64

Now after creating the dummy variable and checking the counts, I move onto visualizing the counts of add ons over the years. During my first iteration of creating the graph, it was far too noisy to make any sense. I created a smoothing function by taking the moving average of thirty days. This will help reduce the noise by taking the average of the specified time period rather than the pure counts.

Below is the graph of the counts of add ons over time. We can see that the moving averages (orange) is much smoother than the raw counts (blue). From here, it doesn't look like there is much of a change, although the averages towards the end of this year seems to be dropping down.

In [ ]:
addons = merged.ix[merged['add_ons'] == 1, :].groupby('date').count()['add_ons']
rolling = pd.Series(addons)
rolling = rolling.rolling(window=30).mean()
trace = go.Scatter(
    x = addons.index.tolist(),
    y = addons.values.tolist(),
    mode = 'lines',
    name = 'Add Ons'
)
trace1 = go.Scatter(
    x = addons.index.tolist(),
    y = rolling.values.tolist(),
    mode = 'lines',
    name = 'Add Ons Smoothed'
)
layout = dict(title = 'Addons Over Time',
              xaxis = dict(title='Month'),
              yaxis = dict(title='Number of Addons Sold, Smoothed'),
              )

In [20]:
# fig = dict(data=[trace, trace1], layout=layout)
# py.iplot(fig, filename='line-mode')
html = '''
<iframe width="900" height="600" frameborder="0" scrolling="no" src="//plot.ly/~minh5/316.embed"></iframe>
'''
display(HTML(html))

# Does the seasonality affect sales?

My first job was at Auntie Anne's and then I spent most of college waiting tables to make money. I know first-hand that the weather can affect sales. I knew that lemonade and regular pretzels sales will skyrocket during the Summer and that cinnamon pretzels are popular during the Winter.

To test this, I used a Bayesian approach, which was inspired by [this book](http://camdavidsonpilon.github.io/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/). The idea of Bayesian perspective is that the data is fixed and the parameters of the distribution need to be determined, while the Frequentist approach assumed the parameter of a distribution to be fixed and the data is random. For instance, most hypothesis testing would assume the data is drawn from a certain distribution and would test results to determine if the data presented is from that same underlying distribution. In this case, the counts of the items sold can be modeled from a Poisson distribution, which takes in a parameter, alpha, which represents the expected count. To determine the paramter, I made another assumption: that there is a point where the distribution that represents the sale of a certain item changes. In this instance, I assumed the cut off point to be summer and winter (because, let's be honest, those are the only two seasons we get in DC). 

First, I separated the data set by the fiscal year, 2013, 2014, and 2015. Then, I separated the data set into only salad sales, since I assumed that salads have seasonal components. For instance, salads might be more popular in the summer and bowls could be more popular in the winter (when people aren't so conscious about their bodies).


In [30]:
fy_13 = merged.ix[merged['date'] < datetime.datetime(2014, 1, 1), :]
fy_14 = merged.ix[((merged['date'] >= datetime.datetime(2014, 1, 1)) & (merged['date'] < datetime.datetime(2015, 1, 1))), :]
fy_15 = merged.ix[merged['date'] >= datetime.datetime(2014, 1, 1), :]

In [89]:
fy_13_salad = fy_13.ix[fy_13['combo'].str.contains('Salad') , :].groupby('date').count()['combo']
fy_14_salad = fy_14.ix[fy_14['combo'].str.contains('Salad') , :].groupby('date').count()['combo']
fy_15_salad = fy_15.ix[fy_15['combo'].str.contains('Salad') , :].groupby('date').count()['combo']

I then created a probabilistic model that tries to determine the where there are points (switch points) in the data that are different. This is represented by the variable, tau, which is drawn from a discrete uniform distribution. This is because I have no assumption of where the switch point is (because DC weather is so variable, my guess is as good as anybody of when does Winter ends and Summer begins). And before and after this switch point, the count data can be modeled by the Poisson distribution, which as I mentioned earlier, which has the parameter, lambda. In this model, lambda is drawn from an exponential distribution, which takes in positive numbers. This model will iterate over and assign different lambda values depending on which season it is on. After this, it will create a "posterior" distribution, which represent the distribution of the data after determining the switch points and assigning the lambda parameters. Then we can employ Markov Chain Monte Carlo to draw sample from this distribution so we can have an idea of what the sales look like. I wrote a quick function below and used salad sales from 2013 as an example.

In [139]:
def create_seasonality_chart(data, num_samples=10000,
                             dist_title='Posterior Distribution',
                             days_title='Distribution of Sales Change Days'):
    with pm.Model() as model:
        alpha = 1.0/data.mean()
        lambda_1 = pm.Exponential("lambda_1", alpha)
        lambda_2 = pm.Exponential("lambda_2", alpha)
        tau = pm.DiscreteUniform("tau", lower=0, upper=len(data) - 1)
        idx = np.arange(len(data))
        lambda_ = pm.math.switch(tau >= idx, lambda_1, lambda_2)
        observation = pm.Poisson("obs", lambda_, observed=data)
        step = pm.Metropolis()
        trace = pm.sample(num_samples, tune=5000, step=step)

    lambda_1_samples = trace['lambda_1']
    lambda_2_samples = trace['lambda_2']
    tau_samples = trace['tau']

    trace1 = go.Histogram(
        x=lambda_1_samples,
        opacity=0.75,
        name='Winter'
    )
    trace2 = go.Histogram(
        x=lambda_2_samples,
        opacity=0.75,
        name='Summer'
    )

    graph_data = [trace1, trace2]
    layout = go.Layout(barmode='overlay', title=dist_title)
    fig = go.Figure(data=graph_data, layout=layout)
    
    trace_days = go.Histogram(
    x=tau_samples,
    opacity=0.75,
    name='Days'
    )
    days_data = [trace_days]
    layout = go.Layout(barmode='overlay', title=days_title)
    days_fig = go.Figure(data=days_data, layout=layout)

    return fig, days_fig

In [14]:
# fig1, fig2 = create_seasonality_chart(fy_13_salad, dist_title='Posterior Distribution of 2013 Salads Sales')
# py.iplot(fig1, filename='fy_13_salad')
html = '''
<iframe width="900" height="600" frameborder="0" scrolling="no" src="//plot.ly/~minh5/387.embed"></iframe>
'''
display(HTML(html))

In [12]:
# py.iplot(fig2, filename='fy_13_salad_days')
html = '''
<iframe width="900" height="400" frameborder="0" scrolling="no" src="//plot.ly/~minh5/389.embed"></iframe>
'''
display(HTML(html))

From this data, we can see two distinct distribution. We can see that average salad sale in the Winter is around 440 while it looks to be around 450, which seems to be higher. And we can look at the second plot, we see where the switch point occurs, where we see that the salad sales changed, around 177 days (6 months), which seems a reasonable time for summer to begin in DC. 

Now the data is not always this clear cut. We can look at the salad sales data for 2014 and 2015.

In [16]:
# fig1, fig2 = create_seasonality_chart(fy_14_salad, dist_title='Posterior Distribution of 2014 Salads')
# py.iplot(fig1, filename='fy_14_salad_days')
html = '''
<iframe width="900" height="600" frameborder="0" scrolling="no" src="//plot.ly/~minh5/391.embed"></iframe>
'''
display(HTML(html))

In [18]:
# fig1, fig2 = create_seasonality_chart(fy_15_salad, dist_title='Posterior Distribution of 2015 Salads')
# py.iplot(fig1, filename='fy_15_salad_days')
html = '''
<iframe width="900" height="600" frameborder="0" scrolling="no" src="//plot.ly/~minh5/393.embed"></iframe>
'''
display(HTML(html))

These two distributions are much "fatter" than the 2013, which means that the level of uncertainty is much higher. In 2014, we see that the two distribution overlaps a bit more than 2013. In 2015, we see an odd outcome, the distribution for the Summer is engulfed inside the Winter sales. The Winter sales are very wide, which means that model is unable to determine the lambda for the salad sales. Unlike the 2013 salad sales, the model is unable to determine any seasonality for the sales.

# Conclusion

For this exercise, my main concern is to just understand the data, come up with hypothesis, and think about how I would go about in solving it. The questions I tried to test were:
  - What foods are usually paired with each other?
  - What relationship do the foods have to each other?
  - Is there a seasonality effect on food choices?
  
I found out that the food item that is the most sold are the Chicken salad and the chicken bowl. I found out that chicken bowl is **slightly** correlated with the chicken salad and greens and grains. Along the way, I explored two more questions:
  - What about transactions that have more than one entree?
  - What about transactions with add ons?
  
During this, I discovered out that there is a sizeable number of customers who buy the same entrees, which could be a useful business straetegy. Regarding add ons, we see that there is not much of a change in the number of add ons sold over the years. Finally, when looking at seasonality, I used a Bayesian method to determine if the distribution between the two seasons were different. I showed that the distribution for salad sales in 2013 have a higher chance of being different.
These approaches can be reused over and over to test on different aspects of the data that I did not have a chance to explore. Ideally, if I had the time, I would examine every single food base and protein to mine for insights that **(omitted)** could use to gain new customers and retain current ones.